In [1]:
import pandas as pd
import numpy as np

# **TRAIN DATA**

In [2]:
x_train=np.array([[0,25,2,0,0,0],
                  [1,30,2,0,1,0],
                  [2,35,1,0,0,1],
                  [3,40,0,0,0,1],
                  [4,45,0,1,0,1],
                  [5,50,0,1,1,0],
                  [6,55,1,1,1,1],
                  [7,60,2,0,0,0]])

# **GET THE POTENTIAL SPLIT**

In [3]:
def get_potential_split(data, indices):
    data = data[indices]
    data = data[data[:, 1].argsort()]
    label_col1 = data[:, 1]  # Age
    label_col2 = data[:, 2]  # Income
    label_col3 = data[:, 3]  # Student
    label_col4 = data[:, 4]  # Credit rating

    age_cut = [(label_col1[i] + label_col1[i+1])/2 for i in range(len(label_col1)-1)] if len(label_col1) > 1 else []
    income_cut = list(set(label_col2)) if len(label_col2) > 0 else []
    student_cut = list(set(label_col3)) if len(label_col3) > 0 else []
    credit_rating = list(set(label_col4)) if len(label_col4) > 0 else []

    return np.array(age_cut), np.array(income_cut), np.array(student_cut), np.array(credit_rating)



# **GINI INDEX HELPER FN**

In [4]:
def gini(labels):
    """Calculate Gini index using ∑ Pmk(1-Pmk)."""
    class_labels, counts = np.unique(labels, return_counts=True)  # Count occurrences of each class
    probabilities = counts / counts.sum()  # Compute class probabilities

    gini = np.sum(probabilities * (1 - probabilities))  # ∑ Pmk(1-Pmk)

    return gini

# **GINI INDEX**

In [5]:
def gini_index(left_labels, right_labels):
    """Calculate the weighted Gini index for a split."""
    total_samples = len(left_labels) + len(right_labels)

    gini_left =gini(left_labels)
    gini_right =gini(right_labels)

    weighted_gini = (len(left_labels) / total_samples) * gini_left + (len(right_labels) / total_samples) * gini_right

    return weighted_gini

# **FINDING THE BEST SPLIT**

In [6]:
def find_best_split(data, indices, age_splits, income_splits, student_splits, credit_splits):
    """Finds the best split feature and value that minimizes Gini index."""
    newdata = data[indices]  # Get only relevant data points

    best_gini = float('inf')  #INFINITY intialise
    best_split = None  #intally null

    # Loop separately for each feature

    #  Loop for Age
    if len(age_splits)>0:
      for threshold in age_splits: # loop on all the thresold point of age split
          left_indices = indices[newdata[:, 1] <= threshold]# applying the mask
          right_indices = indices[newdata[:, 1] > threshold]

          left_labels = data[left_indices][:, -1]
          right_labels = data[right_indices][:, -1]

          gini = gini_index(left_labels, right_labels)

          if gini < best_gini:
              best_gini = gini
              best_split = (1, threshold, left_indices, right_indices)

    #  Loop for Income
    if len(income_splits)>0:
      for threshold in income_splits:
          left_indices = indices[newdata[:, 2] == threshold]
          right_indices = indices[newdata[:, 2] != threshold]

          left_labels = data[left_indices][:, -1]
          right_labels = data[right_indices][:, -1]

          gini = gini_index(left_labels, right_labels)

          if gini < best_gini:
              best_gini = gini
              best_split = (2, threshold, left_indices, right_indices)

    #  Loop for Student type
    if len(student_splits)>0:
      for threshold in student_splits:
          left_indices = indices[newdata[:, 3] == threshold]
          right_indices = indices[newdata[:, 3] != threshold]

          left_labels = data[left_indices][:, -1]
          right_labels = data[right_indices][:, -1]

          gini = gini_index(left_labels, right_labels)

          if gini < best_gini:
              best_gini = gini
              best_split = (3, threshold, left_indices, right_indices)

    #  Loop for Credit Rating
    if len(credit_splits)>0:
      for threshold in credit_splits:
          left_indices = indices[newdata[:, 4] == threshold]
          right_indices = indices[newdata[:, 4] != threshold]

          left_labels = data[left_indices][:, -1]
          right_labels = data[right_indices][:, -1]

          gini = gini_index(left_labels, right_labels)

          if gini < best_gini:
              best_gini = gini
              best_split = (4, threshold, left_indices, right_indices)



    return best_split  # (best feature, best threshold, left indices, right indices)


# **GEETING THE PURITY**

In [7]:
def purity(data):
 label_col=data[:,-1]
 unique_labels=np.unique(label_col)
 if(len(unique_labels)==1):
  return True
 else:
  return False

# **CREATING THE NODE CLASS**

In [8]:
import numpy as np

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, leaf_indices=None):
        self.feature = feature  # Feature used for splitting
        self.threshold = threshold  # Threshold value
        self.left = left  # Left subtree
        self.right = right  # Right subtree
        self.leaf_indices = leaf_indices  # Indices of samples in this leaf (if it's a leaf node)

    def is_leaf(self):
        return self.leaf_indices is not None  # node is a leaf if it has leaf_indices


# **CREATING THE TREE**

In [9]:
max_depth=2
def building_the_tree(data, indices, depth=0):
    if depth == max_depth or len(indices) <= 2 or purity(data[indices]):
        return Node(leaf_indices=list(map(int, indices)))

    age_splits, income_splits, student_splits, credit_splits = get_potential_split(data, indices)
    best_split = find_best_split(data, indices, age_splits, income_splits, student_splits, credit_splits)

    if best_split is None:
        return Node(leaf_indices=list(map(int, indices)))

    best_feature, best_threshold, left_indices, right_indices = best_split
    print(f"Depth {depth}: Best Split → Feature={best_feature}, Threshold={best_threshold}")

    left_subtree = building_the_tree(data, left_indices, depth+1)
    right_subtree = building_the_tree(data, right_indices, depth+1)

    return Node(feature=best_feature, threshold=best_threshold, left=left_subtree, right=right_subtree)

# **DECISION TREE BULID ON WHOLE TRAINING SAMPLE**

# **BUILDING THE TREE WITH THE TRAIN DATA**

In [10]:
n=building_the_tree(x_train, x_train[:,0], 0)

Depth 0: Best Split → Feature=2, Threshold=2
Depth 1: Best Split → Feature=1, Threshold=47.5


# **PREDICTING WITH A SAMPLE ON THE BASIS OF ABOVE TREE: this gives leaf node to which sample belong to**

In [11]:
def predict_leaf_node(node, sample):
    """Traverses the decision tree to find the leaf node for a given sample."""
    if node.is_leaf():
        return node.leaf_indices  # Return leaf indices if it's a leaf
    if node.feature==1:
      if sample[node.feature] <= node.threshold:
          return predict_leaf_node(node.left, sample)  # Go left
      else:
          return predict_leaf_node(node.right, sample)  # Go right
    else:
      if sample[node.feature] == node.threshold:
          return predict_leaf_node(node.left, sample)  # Go left
      else:
          return predict_leaf_node(node.right, sample)  # Go right


> * # THE FEATURES ARE AT INDEX 1,2,3,4
> * # INDEX 0 IS FOR INDEX THAT THE SAMPLE HAS IN THE DATASET
> * # INDEX 5 IS FOR ITS LABEL
> * # THATS WHY PASS -1 AT 0 index AND -1 at last IF NEW SAMPLE COMES

In [12]:
i=(predict_leaf_node(n,np.array([-1,42,0,0,1,-1])))
print("By tree predicted sample would belong to ",i)

By tree predicted sample would belong to  [2, 3, 4]


# **Return the decision of the tree**

In [13]:
def result(x_train,i):
  x_train[i][:,-1]
  hashmap={}
  for i in x_train[i][:,-1]:
      if i not in hashmap:
        hashmap[i]=0
      hashmap[i]=hashmap[i]+1
  maxi=float('-inf')
  ans=None
  for key, value in hashmap.items():
      if(value>maxi):
        ans=key
  return(key)

In [14]:
print("Decision of tree trained on whole dataset",result(x_train,i))

Decision of tree trained on whole dataset 1


# **Genrating 10 bagged dataset**

In [15]:
import numpy as np
from sklearn.utils import resample


bagged_datasets = []
oob_for_each_dataset=[]
# Generate 10 bagged datasets
for _ in range(10):
  #these indices are taken from the original sample data
    original_indices=np.array([0,1,2,3,4,5,6,7])
    X_sample,sample_indices = resample(x_train,original_indices, n_samples=8, replace=True)  # Bootstrap sampling
    # Find Out-of-Bag (OOB) samples
    oob_indices = np.setdiff1d(original_indices, sample_indices)
    oob_for_each_dataset.append(oob_indices)
    for i in range(len(X_sample)):
      X_sample[i][0]=i
    bagged_datasets.append(X_sample)

# Print the bagged datasets
for i, dataset in enumerate(bagged_datasets):
    print(f"Bagged Dataset {i+1}:\n{dataset}\n")


Bagged Dataset 1:
[[ 0 60  2  0  0  0]
 [ 1 50  0  1  1  0]
 [ 2 35  1  0  0  1]
 [ 3 50  0  1  1  0]
 [ 4 60  2  0  0  0]
 [ 5 30  2  0  1  0]
 [ 6 40  0  0  0  1]
 [ 7 45  0  1  0  1]]

Bagged Dataset 2:
[[ 0 55  1  1  1  1]
 [ 1 25  2  0  0  0]
 [ 2 40  0  0  0  1]
 [ 3 40  0  0  0  1]
 [ 4 30  2  0  1  0]
 [ 5 50  0  1  1  0]
 [ 6 35  1  0  0  1]
 [ 7 45  0  1  0  1]]

Bagged Dataset 3:
[[ 0 30  2  0  1  0]
 [ 1 25  2  0  0  0]
 [ 2 45  0  1  0  1]
 [ 3 50  0  1  1  0]
 [ 4 45  0  1  0  1]
 [ 5 45  0  1  0  1]
 [ 6 30  2  0  1  0]
 [ 7 45  0  1  0  1]]

Bagged Dataset 4:
[[ 0 30  2  0  1  0]
 [ 1 50  0  1  1  0]
 [ 2 30  2  0  1  0]
 [ 3 30  2  0  1  0]
 [ 4 25  2  0  0  0]
 [ 5 45  0  1  0  1]
 [ 6 50  0  1  1  0]
 [ 7 40  0  0  0  1]]

Bagged Dataset 5:
[[ 0 45  0  1  0  1]
 [ 1 40  0  0  0  1]
 [ 2 25  2  0  0  0]
 [ 3 40  0  0  0  1]
 [ 4 25  2  0  0  0]
 [ 5 50  0  1  1  0]
 [ 6 60  2  0  0  0]
 [ 7 35  1  0  0  1]]

Bagged Dataset 6:
[[ 0 30  2  0  1  0]
 [ 1 30  2  0  1  0]


In [16]:
oob_for_each_dataset

[array([0, 6]),
 array([7]),
 array([2, 3, 6, 7]),
 array([2, 6, 7]),
 array([1, 6]),
 array([2, 3, 4]),
 array([1, 4, 6, 7]),
 array([1, 2, 7]),
 array([0, 3, 7]),
 array([0, 1, 2, 3, 6])]

In [17]:
def counter(prediction):

  count_dict = {}
  for pred in predictions:
      count_dict[pred] = count_dict.get(pred, 0) + 1


  final_prediction = max(count_dict, key=count_dict.get)

  return(final_prediction)


In [18]:
import numpy as np

# List to hold the 10 trees
nodes = []
# List to collect predictions from each tree for the fixed sample/to be predicted sample
predictions = []

oob_error=[]

fixed_sample = np.array([-1, 42, 0, 0, 1, -1])  # fixed sample for prediction

# Build trees using the 10 bagged datasets
for i in range(10):
    tree = building_the_tree(bagged_datasets[i], bagged_datasets[i][:, 0], 0)
    nodes.append(tree)

# Get the prediction from each tree for the fixed sample
for i in range(10):
    # Get the leaf node prediction for the fixed sample from the i-th tree
    leaf_pred = predict_leaf_node(nodes[i], fixed_sample)
    # Convert the leaf node prediction into a class label/class predicted
    class_pred = result(bagged_datasets[i], leaf_pred)
    predictions.append(class_pred)
    for f in oob_for_each_dataset[i]:
      leaf_pred = predict_leaf_node(nodes[i], x_train[f])
      class_pred = result(bagged_datasets[i], leaf_pred)
      oob_error.append(class_pred-x_train[f][-1])

#final result
final_oob_error=0
count=0
for h in oob_error:
  final_oob_error=final_oob_error+pow(h,2)
  count=count+1
final_oob_error=final_oob_error/count
print("final oob error is:",final_oob_error)
# Final prediction by majority voting
final_prediction = counter(predictions)
print("Final prediction (majority vote) is:", final_prediction)


Depth 0: Best Split → Feature=1, Threshold=47.5
Depth 1: Best Split → Feature=1, Threshold=32.5
Depth 0: Best Split → Feature=1, Threshold=32.5
Depth 1: Best Split → Feature=1, Threshold=47.5
Depth 0: Best Split → Feature=1, Threshold=30.0
Depth 1: Best Split → Feature=1, Threshold=45.0
Depth 0: Best Split → Feature=4, Threshold=0
Depth 1: Best Split → Feature=1, Threshold=32.5
Depth 0: Best Split → Feature=2, Threshold=2
Depth 1: Best Split → Feature=1, Threshold=47.5
Depth 0: Best Split → Feature=2, Threshold=1
Depth 0: Best Split → Feature=1, Threshold=25.0
Depth 1: Best Split → Feature=1, Threshold=40.0
Depth 0: Best Split → Feature=1, Threshold=25.0
Depth 1: Best Split → Feature=1, Threshold=52.5
Depth 0: Best Split → Feature=1, Threshold=30.0
Depth 1: Best Split → Feature=1, Threshold=47.5
Depth 0: Best Split → Feature=1, Threshold=45.0
final oob error is: 0.5
Final prediction (majority vote) is: 1


# **BUILDING THE RANDOM FOREST**

In [19]:
max_depth = 2
import random
def building_the_random_forest_tree(data, indices, depth):
    if depth == max_depth or len(indices) <= 2 or purity(data[indices]):
        return Node(leaf_indices=list(map(int, indices)))

    age_splits, income_splits, student_splits, credit_splits = get_potential_split(data, indices)

    # Corrected: Map features to their actual column indices (1-4)
    available_features = [1, 2, 3, 4]  # Age (col1), Income (col2), Student (col3), Credit (col4)
    selected_features = random.sample(available_features, 2)  # Now correctly picks 2 actual features
    print(f"selected feature,{selected_features}")
    # Disable non-selected features by emptying their splits
    if 1 not in selected_features: age_splits = []
    if 2 not in selected_features: income_splits = []
    if 3 not in selected_features: student_splits = []
    if 4 not in selected_features: credit_splits = []

    best_split = find_best_split(data, indices, age_splits, income_splits, student_splits, credit_splits)

    if best_split is None:
        return Node(leaf_indices=list(map(int, indices)))

    best_feature, best_threshold, left_indices, right_indices = best_split
    print(f"Depth {depth}: Best Split → Feature={best_feature}, Threshold={best_threshold}")

    left_subtree = building_the_random_forest_tree(data, left_indices, depth+1)
    right_subtree = building_the_random_forest_tree(data, right_indices, depth+1)

    return Node(feature=best_feature, threshold=best_threshold, left=left_subtree, right=right_subtree)

# **Genrating Bagged dataset for random forest**

In [20]:
import numpy as np
from sklearn.utils import resample


bagged_datasets_for_rf = []
oob_for_each_dataset_rf=[]

# Generate 10 bagged datasets
for _ in range(10):
  #these indices are taken from the original sample data
    original_indices=np.array([0,1,2,3,4,5,6,7])
    X_sample,sample_indices = resample(x_train,original_indices, n_samples=8, replace=True)  # Bootstrap sampling
    # Find Out-of-Bag (OOB) samples
    oob_indices = np.setdiff1d(original_indices, sample_indices)
    oob_for_each_dataset_rf.append(oob_indices)
    for i in range(len(X_sample)):
      X_sample[i][0]=i
    bagged_datasets_for_rf.append(X_sample)

# Print the bagged datasets
for i, dataset in enumerate(bagged_datasets_for_rf):
    print(f"Bagged Dataset {i+1}:\n{dataset}\n")

Bagged Dataset 1:
[[ 0 25  2  0  0  0]
 [ 1 25  2  0  0  0]
 [ 2 45  0  1  0  1]
 [ 3 45  0  1  0  1]
 [ 4 25  2  0  0  0]
 [ 5 35  1  0  0  1]
 [ 6 40  0  0  0  1]
 [ 7 35  1  0  0  1]]

Bagged Dataset 2:
[[ 0 55  1  1  1  1]
 [ 1 60  2  0  0  0]
 [ 2 45  0  1  0  1]
 [ 3 45  0  1  0  1]
 [ 4 35  1  0  0  1]
 [ 5 50  0  1  1  0]
 [ 6 45  0  1  0  1]
 [ 7 25  2  0  0  0]]

Bagged Dataset 3:
[[ 0 45  0  1  0  1]
 [ 1 45  0  1  0  1]
 [ 2 45  0  1  0  1]
 [ 3 40  0  0  0  1]
 [ 4 30  2  0  1  0]
 [ 5 25  2  0  0  0]
 [ 6 45  0  1  0  1]
 [ 7 30  2  0  1  0]]

Bagged Dataset 4:
[[ 0 35  1  0  0  1]
 [ 1 55  1  1  1  1]
 [ 2 45  0  1  0  1]
 [ 3 55  1  1  1  1]
 [ 4 60  2  0  0  0]
 [ 5 30  2  0  1  0]
 [ 6 40  0  0  0  1]
 [ 7 30  2  0  1  0]]

Bagged Dataset 5:
[[ 0 50  0  1  1  0]
 [ 1 50  0  1  1  0]
 [ 2 30  2  0  1  0]
 [ 3 40  0  0  0  1]
 [ 4 40  0  0  0  1]
 [ 5 60  2  0  0  0]
 [ 6 60  2  0  0  0]
 [ 7 50  0  1  1  0]]

Bagged Dataset 6:
[[ 0 45  0  1  0  1]
 [ 1 40  0  0  0  1]


In [21]:
import numpy as np

# List to hold the 10 trees
nodes_rf = []
# List to collect predictions from each tree for the fixed sample
predictions_rf = []

#oob error store
oob_error_rf=[]

fixed_sample_rf = np.array([-1, 42, 0, 0, 1, -1])  # fixed sample for prediction

# Build trees using the 10 bagged datasets
for i in range(10):
    tree = building_the_random_forest_tree(bagged_datasets_for_rf[i], bagged_datasets_for_rf[i][:, 0], 0)
    nodes_rf.append(tree)

# Get the prediction from each tree for the fixed sample
for i in range(10):
    # Get the leaf node prediction for the fixed sample from the i-th tree
    leaf_pred = predict_leaf_node(nodes_rf[i], fixed_sample_rf)
    # Convert the leaf node prediction into a class label
    class_pred = result(bagged_datasets_for_rf[i], leaf_pred)
    predictions_rf.append(class_pred)
    for f in oob_for_each_dataset_rf[i]:
      leaf_pred = predict_leaf_node(nodes_rf[i], x_train[f])
      class_pred = result(bagged_datasets_for_rf[i], leaf_pred)
      oob_error_rf.append(class_pred-x_train[f][-1])

#final result
final_oob_error_rf=0
count=0
for h in oob_error_rf:
  final_oob_error_rf=final_oob_error_rf+pow(h,2)
  count=count+1
final_oob_error_rf=final_oob_error_rf/count
print("final oob error is:",final_oob_error_rf)
# Final prediction by majority voting
final_prediction_rf = counter(predictions_rf)
print("Final prediction (majority vote) is:", final_prediction_rf)


selected feature,[2, 4]
Depth 0: Best Split → Feature=2, Threshold=2
selected feature,[4, 2]
Depth 0: Best Split → Feature=2, Threshold=2
selected feature,[2, 4]
Depth 1: Best Split → Feature=4, Threshold=0
selected feature,[1, 3]
Depth 0: Best Split → Feature=1, Threshold=30.0
selected feature,[1, 2]
Depth 0: Best Split → Feature=2, Threshold=2
selected feature,[1, 2]
Depth 0: Best Split → Feature=1, Threshold=40.0
selected feature,[1, 4]
Depth 1: Best Split → Feature=1, Threshold=35.0
selected feature,[4, 1]
Depth 0: Best Split → Feature=1, Threshold=32.5
selected feature,[4, 3]
Depth 0: Best Split → Feature=4, Threshold=0
selected feature,[1, 4]
Depth 1: Best Split → Feature=1, Threshold=50.0
selected feature,[2, 3]
Depth 0: Best Split → Feature=2, Threshold=2
selected feature,[2, 1]
Depth 1: Best Split → Feature=1, Threshold=45.0
selected feature,[2, 1]
Depth 0: Best Split → Feature=1, Threshold=25.0
selected feature,[4, 2]
Depth 0: Best Split → Feature=2, Threshold=2
final oob err